In [4]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import numpy as np

In [5]:
driver = webdriver.Chrome('c:/chromedriver.exe')
# url = 'file:///C:/Users/Playdata/Desktop/shinhancard_20200910_900046432/shinhancard_20200910_900046432.html'
url = 'file:///C:/Users/hyb95_000/Desktop/%EC%B2%B4%ED%81%AC%EC%B9%B4%EB%93%9C%20%EB%AA%85%EC%84%B8%EC%84%9C/shinhancard_20201110_900046432.html'
driver.get(url)
html = driver.page_source

soup = BeautifulSoup(html,'html.parser')

tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

for n in range(len(tables_middleSum )) :
    tables.remove(tables_middleSum[n])

# 명세서에서 추출한 데이터 string으로 변환하여 분리
seperate_tables = []                          
for n in range(len(tables)) :
    table = str(tables[n].text).split('\n')
    seperate_tables = seperate_tables + table

# seperate_tables에서 '사용날짜' 추출   
date_list = []
for n in range(1,len(seperate_tables), 12) :
    date = seperate_tables[n]
    date_list.append(date)

# seperate_tables에서 '사용처' 데이터만 추출    
store_list = []
for n in range(3,len(seperate_tables),12) :
    store = seperate_tables[n].replace(" ", "")
    store_list.append(store)

# 카카오 API활용하여 seperate_tables에서 '사용처 주소' 데이터만 추출    
location_list = []
def create_url(search_text):
    headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}

    base = "https://dapi.kakao.com/v2/local/search/keyword.json"
    param_query = "?query=" +search_text
#     print(f'base + param_query: {base + param_query}')
    respond = requests.get(base + param_query,headers=headers)
    return respond.json()

for n in range(len(store_list)) :
# for n in range(10) :
    store_name = store_list[n]
    result = create_url(store_name)
    tmp1 = result['documents']
    try :
        tmp2 = tmp1[0]
        tmp3 = tmp2['address_name']
        location_list.append(tmp3)
    except :
        tmp2 = np.nan
        location_list.append(tmp2)

# seperate_tables에서 '사용금액' 데이터만 추출  
price_list = [] 
for n in range(4,len(seperate_tables),12) :
    price = seperate_tables[n]
    price_list.append(price)

# 추출한 데이터로 DataFrame 변환
table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
df1 = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])

driver.close()

df1

,사용날짜,사용처,사용처주소,사용금액
0,20.09.27,서호김밥,서울 서초구 방배동 790-13,"14,000"
1,20.09.27,GS25성수포레점,서울 성동구 성수동1가 685-696,"10,000"
2,20.09.28,왕창할인마트,서울 동작구 사당동 127-9,"5,700"
3,20.10.02,주식회사세녹코리아,NaN,"5,500"
4,20.10.02,주식회사아성다이소,경남 창원시 성산구 상남동 26-4,"6,000"
...,...,...,...,...
84,20.10.22,(주)티머니_지하철,NaN,"1,250"
85,20.10.23,(주)티머니_지하철,NaN,"1,250"
86,20.10.23,(주)티머니_지하철,NaN,"1,250"
87,20.10.24,(주)티머니_지하철,NaN,"1,250"


In [15]:
# 위치정보 없는 데이터 삭제한 DF 만들기
df_address = df1.dropna()

# 중복 사용지점 없는 DF 만들기
df_address_only = df_address.drop_duplicates(subset = '사용처')

name_index =  df_address['사용처']
data_in_name = df_address.rename(name_index)
data_in_name = data_in_name.drop(['사용처'], axis = 1)

name_index =  df_address_only['사용처']
data_in_only_name = df_address_only.rename(name_index)
data_in_only_name = data_in_only_name.drop(['사용처'], axis = 1)

for name in list(name_index):
    tmp_data = data_in_name.loc[name]['사용금액']

    if type(tmp_data) != str:
        price = [price.replace(',','') for price in tmp_data]
        price = list(map(int, price))
        price_sum = sum(price)
        data_in_only_name.loc[name]['사용금액'] = price_sum
        
    else:
        data_in_only_name.loc[name]['사용금액'] = int(tmp_data.replace(',',''))
        
name_series = df_address['사용처']

visit_count = dict(name_series.value_counts())

count_list = []

for k in name_index:
    count_list.append(visit_count[k])
    
# 사용횟수 열 추가하기

data_in_only_name.insert(3, '방문횟수', count_list)
data_in_only_name

,사용날짜,사용처주소,사용금액,방문횟수
서호김밥,20.09.27,서울 서초구 방배동 790-13,14000,1
GS25성수포레점,20.09.27,서울 성동구 성수동1가 685-696,10000,1
왕창할인마트,20.09.28,서울 동작구 사당동 127-9,29000,4
주식회사아성다이소,20.10.02,경남 창원시 성산구 상남동 26-4,6000,1
씨제이올리브영(주)방배이수점,20.10.03,서울 서초구 방배동 1671,3600,1
북창동순두부(서초점),20.10.05,서울 서초구 서초동 1443-11,8500,1
항아리식당,20.10.06,서울 서초구 서초동 1439-14,8000,1
(주)무신사,20.10.06,서울 강남구 신사동 640-2,52707,1
송황칼국수,20.10.07,서울 용산구 청파동2가 96-1,14000,2
GS25서초프랜드점,20.10.07,서울 서초구 서초동 1605-4,3500,1


In [33]:
date = list(data_in_only_name['사용날짜'])
store_name = list(name_index)
address = list(data_in_only_name['사용처주소'])
price = list(data_in_only_name['사용금액'])
count = count_list

In [25]:
# # 주소 있는 요소 인덱스만 반환
# i = 0
# loc_index = []
# for name in location_list:
    
#     if(name != False):
#         loc_index.append(i)
        
#     i = i + 1
    

# # 주소 없는 요소 삭제
# # while False in location_list:
# #     location_list.remove(False)

In [ ]:
# # DataFrame Excel로변환
# excel_writer = pd.ExcelWriter('C:/Users/hyb95_000/Desktop/체크카드 명세서/carduselist.xlsx', 
#                               engine='xlsxwriter')
# df1.to_excel(excel_writer, index = False, sheet_name='카드사용리스트_09월')
# excel_writer.save()

In [18]:
import requests

# date = date_list
# store_name = store_list
# address = location_list
# price = price_list


# address = ['동작대로 29길 91', '서초구 효령로 335']
coordinates_x = []
coordinates_y = []
# store_name = ['우리집', '플레이데이터']

# price = ['10000', '2000']
 
def getLatLng(address):
    result = ""
 
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
    rest_api_key = '289858dc68417f30677b064987b71bab'
    header = {'Authorization': 'KakaoAK ' + rest_api_key}
 
    r = requests.get(url, headers=header)
 
    if r.status_code == 200:
        result_address = r.json()["documents"][0]["address"]
        
        result = result_address["y"], result_address["x"]
    else:
        result = "ERROR[" + str(r.status_code) + "]"
    
    return result
 
    
# 카카오 REST API로 좌표 구하기
for j in address:
    
    if j != False:
        coordinates_x.append(getLatLng(j)[0])
        coordinates_y.append(getLatLng(j)[1])
    else:
        coordinates_x.append('')
        coordinates_y.append('')
        
    
# n = len(address)

# center_x_tmp = .0
# center_y_tmp = .0

# for k in range(len(store_name)):
#         center_x_tmp = center_x_tmp + float(coordinates_x[k])
#         center_y_tmp = center_y_tmp + float(coordinates_y[k])
    
# center_x = center_x_tmp / n
# center_y = center_y_tmp / n

In [35]:
html = '<div id="map" style="width:100%;height:100%;"></div>'
html = html + '<script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>\n'
html = html + '<script>\n'
html = html + "var mapContainer = document.getElementById('map'), \n"
html = html + '    mapOption = { \n'

html = html + '        center: new kakao.maps.LatLng(37.4994849454133, 127.028098958753), \n'
html = html + '        level: 7 \n'
html = html + '    };'


html = html + """
var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다
 
// 마커를 표시할 위치와 title 객체 배열입니다 

var positions = []
"""
for i in range(len(store_name)):
    html = html + "positions.push({ title: " + f"'{store_name[i]}', price: '{price[i]}', count: '{count[i]}'," + f" latlng: new kakao.maps.LatLng({coordinates_x[i]}, {coordinates_y[i]})" + "})\n"

html = html + """
// 마커 이미지의 이미지 주소입니다
var imageSrc = "https://t1.daumcdn.net/localimg/localimages/07/mapapidoc/markerStar.png"; 
    
for (var i = 0; i < positions.length; i ++) {
    
    // 마커 이미지의 이미지 크기 입니다
    var imageSize = new kakao.maps.Size(24, 35); 
    
    // 마커 이미지를 생성합니다    
    var markerImage = new kakao.maps.MarkerImage(imageSrc, imageSize); 
    
    // 마커를 생성합니다
    var marker = new kakao.maps.Marker({
        map: map, // 마커를 표시할 지도
        position: positions[i].latlng, // 마커를 표시할 위치
        title : positions[i].title, // 마커의 타이틀, 마커에 마우스를 올리면 타이틀이 표시됩니다
        image : markerImage // 마커 이미지 
    });
    
    var infowindow = new kakao.maps.InfoWindow({
        position : positions[i].latlng, 
        content : '<div style="width:100%;height:100%;font-size:11px">상호명 : ' + positions[i].title + '<br>방문횟수 : ' + positions[i].count + '<br>사용금액 : ' + positions[i].price +  '</div>'
    });
    
    
    infowindow.open(map, marker); 
    
}
</script>"""

map_file = open('map.html', 'w')
map_file.write(html)
map_file.close()

print(html)

<div id="map" style="width:100%;height:100%;"></div><script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=562beaefa8f3396ba21741328654bece"></script>
<script>
var mapContainer = document.getElementById('map'), 
    mapOption = { 
        center: new kakao.maps.LatLng(37.4994849454133, 127.028098958753), 
        level: 7 
    };
var map = new kakao.maps.Map(mapContainer, mapOption); // 지도를 생성합니다
 
// 마커를 표시할 위치와 title 객체 배열입니다 

var positions = []
positions.push({ title: '서호김밥', price: '14000', count: '1', latlng: new kakao.maps.LatLng(37.491674275611, 126.986679860698)})
positions.push({ title: 'GS25성수포레점', price: '10000', count: '1', latlng: new kakao.maps.LatLng(37.5457825428325, 127.042478660804)})
positions.push({ title: '왕창할인마트', price: '29000', count: '4', latlng: new kakao.maps.LatLng(37.4883946353717, 126.978344313895)})
positions.push({ title: '주식회사아성다이소', price: '6000', count: '1', latlng: new kakao.maps.LatLng(35.2212533386035, 128.681235260685)})
posit